In [1]:
import pandas as pd
import geopandas as gpd

#, nrows=10000
df = pd.read_csv('Taxi_Trips_2017.csv' )
#df
#for 0 to 5 :
    #df = pd.concat([chunk[chunk['field'] > constant]])

#for community area NaN delete rows

# Drop columns containing null values
#df_dropped_rows = df.dropna(axis=0)

df_trips = df[df['Pickup Community Area'].notna()]
dff_trips = df_trips[df_trips['Dropoff Community Area'].notna()]

dff_trips



,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
6,f00546971a3e4c8ab55761a337443a0dfa33f42f,631de3a4ad35b7d7d75949b33ad59e09820b51b3f58225...,01/01/2017 12:00:00 AM,01/01/2017 12:30:00 AM,1800,0.00,1.703108e+10,1.703108e+10,8.0,8.0,...,1.0,18.50,Credit Card,Taxi Affiliation Services,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)
7,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,b7da4d1247a408c100b4964f236760694290aace75c8b8...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,660,2.70,NaN,NaN,2.0,3.0,...,0.0,12.00,Credit Card,Choice Taxi Association,42.001571,-87.695013,POINT (-87.6950125892 42.001571027),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)
11,149f18c2d8b9bce6001cc9e156092aa562c83bc1,b7a562fe2d37014bb47274cb5baa361b97ff9263e3ba52...,01/01/2017 12:00:00 AM,01/01/2017 12:15:00 AM,120,0.50,NaN,NaN,16.0,16.0,...,0.0,4.50,Cash,Taxi Affiliation Services,41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253)
18,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,56a1119c6ca57e39525cf06829f9ecff553cf4b5ac2482...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,240,1.30,1.703184e+10,1.703183e+10,32.0,28.0,...,0.0,6.25,Cash,Taxi Affiliation Services,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201)
19,f398f04efdae9641df780e0a824770a62ab9a2ea,9149bad3504bb6966a90f2d42d8a98fdd4d8c27bb2e408...,01/01/2017 12:00:00 AM,01/01/2017 12:00:00 AM,420,0.00,1.703128e+10,1.703184e+10,28.0,32.0,...,0.0,6.00,Cash,Blue Ribbon Taxi Association Inc.,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2c132000a1187a934e47415e5979298449846a15,a94a451da8b419ea88e4c4f4a3b097eec3fa290435b992...,01/01/2017 01:45:00 AM,01/01/2017 01:45:00 AM,780,3.60,1.703103e+10,1.703106e+10,3.0,6.0,...,1.0,13.75,Cash,Globe Taxi,41.972668,-87.663865,POINT (-87.6638654956 41.9726679555),41.934659,-87.646730,POINT (-87.6467297286 41.9346591566)
9996,5bbe24625b603be80d53edf601be2992a0a08e0c,83e107c34986e0dc90555d0497d5fd3a82d765cc654c1a...,01/01/2017 01:45:00 AM,01/01/2017 02:00:00 AM,814,1.51,1.703108e+10,1.703184e+10,8.0,32.0,...,1.0,12.25,Credit Card,Chicago Carriage Cab Corp,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707)
9997,f85c85b127dc722bd3b68da137d2c11acf31f609,e509e79f2a107c52325aa45a188582f37b7dc32825c516...,01/01/2017 01:45:00 AM,01/01/2017 01:45:00 AM,300,0.34,1.703108e+10,1.703108e+10,8.0,8.0,...,1.5,6.50,Cash,Patriot Taxi Dba Peace Taxi Associat,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.892073,-87.628874,POINT (-87.6288741572 41.8920726347)
9998,369916e82f59d45c48e5565f9869cc26a9d8211d,0dee7e1a5017191885590af419c6d70b1779ae8dbc8a24...,01/01/2017 01:45:00 AM,01/01/2017 01:45:00 AM,180,1.00,1.703183e+10,1.703106e+10,3.0,6.0,...,2.0,7.25,Cash,Sun Taxi,41.958056,-87.660389,POINT (-87.66038945570001 41.958055933),41.957531,-87.666611,POINT (-87.6666114405 41.957530922)


In [2]:
del dff_trips['Company']
del dff_trips['Payment Type']



In [3]:

import numpy as np
import pandas as pd
import folium
import webbrowser
import os
import math

from h3 import h3
from folium import Map

In [33]:
def create_map(clusters):
    # Create the map object
    map = Map(tiles="cartodbpositron", 
          attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        tooltip = "{0} accidents".format(cluster['count'])
        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#ff0000', 
                                               fill_color='#ff0000', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map)

    # Determine the map bounding box
    max_lat = df_fmap2.Latitude.max()
    min_lat = df_fmap2.Latitude.min()
    max_lon = df_fmap2.Longitude.max()
    min_lon = df_fmap2.Longitude.min()
    
    # Fit the map to the bounds
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map

In [5]:
def show_map(map, file_name):
    map.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)

In [17]:
df_fmap = dff_trips[['Trip ID','Pickup Centroid Latitude','Pickup Centroid Longitude']]
df_fmap

,Trip ID,Pickup Centroid Latitude,Pickup Centroid Longitude
6,f00546971a3e4c8ab55761a337443a0dfa33f42f,41.893216,-87.637844
7,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,42.001571,-87.695013
11,149f18c2d8b9bce6001cc9e156092aa562c83bc1,41.953582,-87.723452
18,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,41.880994,-87.632746
19,f398f04efdae9641df780e0a824770a62ab9a2ea,41.879255,-87.642649
...,...,...,...
9995,2c132000a1187a934e47415e5979298449846a15,41.972668,-87.663865
9996,5bbe24625b603be80d53edf601be2992a0a08e0c,41.902788,-87.626146
9997,f85c85b127dc722bd3b68da137d2c11acf31f609,41.892042,-87.631864
9998,369916e82f59d45c48e5565f9869cc26a9d8211d,41.958056,-87.660389


In [26]:
# data = {'Trip': df_fmap[['Trip ID']], 
#         'Latitude': df_fmap[['Pickup Centroid Latitude']], 
#         'Longitude': df_fmap[['Pickup Centroid Longitude']]}
#df_fmap2 = pd.DataFrame(dff_trips[['Trip ID','Pickup Centroid Latitude','Pickup Centroid Longitude']], columns=['Trip', 'Latitude', 'Longitude'])
#df_fmap2 = pd.DataFrame({"Trip":df_fmap[['Trip ID']], "Latitude":df_fmap[['Pickup Centroid Latitude']],"Longitude":df_fmap[['Pickup Centroid Longitude']]})

df_fmap2 = df_fmap.rename(columns={"Trip ID": "Trip", "Pickup Centroid Latitude": "Latitude","Pickup Centroid Longitude": "Longitude"})

df_fmap2

,Trip,Latitude,Longitude
6,f00546971a3e4c8ab55761a337443a0dfa33f42f,41.893216,-87.637844
7,fe9bfc25af9dffac30370f4e7d57a8c8a8bdbc0d,42.001571,-87.695013
11,149f18c2d8b9bce6001cc9e156092aa562c83bc1,41.953582,-87.723452
18,1b545d0be27b195076e5d2a7d4fc0ebef6ef20ac,41.880994,-87.632746
19,f398f04efdae9641df780e0a824770a62ab9a2ea,41.879255,-87.642649
...,...,...,...
9995,2c132000a1187a934e47415e5979298449846a15,41.972668,-87.663865
9996,5bbe24625b603be80d53edf601be2992a0a08e0c,41.902788,-87.626146
9997,f85c85b127dc722bd3b68da137d2c11acf31f609,41.892042,-87.631864
9998,369916e82f59d45c48e5565f9869cc26a9d8211d,41.958056,-87.660389


In [28]:
df_fmap2 = df_fmap2.loc[df_fmap2.Latitude <=  90.0]
df_fmap2 = df_fmap2.loc[df_fmap2.Latitude >= -90.0]

df_fmap2 = df_fmap2.loc[df_fmap2.Longitude <=  180.0]
df_fmap2 = df_fmap2.loc[df_fmap2.Longitude >= -180.0]
df_fmap2['rad_lng'] = np.radians(df_fmap2['Longitude'].to_numpy())
df_fmap2['rad_lat'] = np.radians(df_fmap2['Latitude'].to_numpy())

h3_level = 11

In [29]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['Latitude'], row['Longitude'], h3_level)

df_fmap2['h3'] = df_fmap2.apply(lat_lng_to_h3, axis=1)


In [31]:
h3_clusters = dict()

for index, row in df_fmap2.iterrows():
    key = row['h3']
    if key in h3_clusters:
        h3_clusters[key]["count"] += 1
    else:
        h3_clusters[key] = {"count": 1,
                            "geom": h3.h3_to_geo_boundary(key)}
        

In [40]:
relevant_clusters = { key:value for (key,value) in h3_clusters.items() }

h3_map = create_map(relevant_clusters)

In [41]:
show_map(h3_map, "map_h3_{0}.html".format(h3_level))